In [1]:
import pickle

import numpy as np
import pandas as pd

from tqdm import tqdm

import evaluate

import torch

import nltk
nltk.download('punkt')

from transformers import (MBartForConditionalGeneration, 
                          MBartTokenizer, 
                          DataCollatorForSeq2Seq,
                          MT5ForConditionalGeneration,
                          MT5Tokenizer,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)

from datasets import load_dataset

2023-12-03 18:02:24.559688: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 18:02:24.585604: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 18:02:24.585626: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 18:02:24.585643: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 18:02:24.590472: I tensorflow/core/platform/cpu_feature_g

# Настройка

In [2]:
# Параметры Bart токенайзера
MAX_LENGTH = 600
PADDING = 'max_length'
TRUNCATION = True
RETURN_TENSORS = 'pt'

# Параметры Bart
BART_MODEL_NAME = "IlyaGusev/mbart_ru_sum_gazeta"
NO_REPEAT_NGRAM = 4

# Параметры T5 токенайзера
MAX_TARGET_TOKENS_COUNT = 128
MAX_SOURCE_TOKENS_COUNT = 1024

# Параметры T5
T5_MODEL_NAME = "google/mt5-small"
OUTPUT_DIR = "T5"
EVALUATION_STRATEGY = "steps"
EVAL_STEPS = 25
LOGGING_STEPS = 25
LEARNING_RATE = 4e-4
PER_DEVICE_TRAIN_BATCH_SIZE = 2
PER_DEVICE_EVAL_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 64
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIMIT = 3
NUM_TRAIN_EPOCHS = 1
FP16 = False
PREDICT_WITH_GENERATE = True
GENERATION_MAX_LENGTH = MAX_TARGET_TOKENS_COUNT
GENERATION_NUM_BEAMS = 5

### Устройство для обучения

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Метрики

In [4]:
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")

# Загрузка датасета

In [5]:
dataset = load_dataset('IlyaGusev/gazeta')

# Обрезка датасета, чтобы на GPU влезло
dataset['train'] = dataset['train'].select(np.random.choice(range(len(dataset['train'])), size = 200))
dataset['test'] = dataset['test'].select(np.random.choice(range(len(dataset['test'])), size = 500))

In [6]:
df = pd.DataFrame(dataset['test'])
df

,text,summary,title,date,url
0,Европейские и американские лошади неоднократно...,Неоднократная миграция и скрещивание европейск...,Ученые выяснили детали древнего скрещивания ев...,2021-05-19 14:23:38,https://www.gazeta.ru/science/2021/05/19_a_135...
1,Основатель Telegram-канала NEXTA Роман Протасе...,Задержанный в Минске основатель Telegram-канал...,Протасевич признал вину и рассказал о подготов...,2021-06-03 23:26:11,https://www.gazeta.ru/politics/2021/06/03_a_13...
2,"Администрация WhatsApp опубликовала статью, в ...",WhatsApp отреагировал на критику своей обновле...,WhatsApp объяснил передачу личных данных польз...,2021-01-12 14:47:06,https://www.gazeta.ru/tech/2021/01/12/13432892...
3,"Остатки вирусных частиц SARS-CoV-2, осевшие в ...",Остатки коронавируса в кишечнике запускают эво...,"Ученые выяснили, как остатки SARS-CoV-2 в кише...",2021-01-22 14:07:16,https://www.gazeta.ru/science/2021/01/22_a_134...
4,Недалеко от деревни Гостилицы в Ломоносовском ...,Менее чем в 50 км от Санкт-Петербурга в воздух...,В Ленобласти разбился легкомоторный самолет с ...,2021-01-08 18:40:53,https://www.gazeta.ru/social/2021/01/08/134295...
...,...,...,...,...,...
495,В преддверии Дня космонавтики Минобороны опубл...,К 60-летию первого полета человека в космос Ми...,Минобороны опубликовало документы о первых кос...,2021-04-10 00:01:40,https://www.gazeta.ru/science/2021/04/09_a_135...
496,Крымское управление ФСБ сообщило об аресте на ...,Подозреваемый в шпионаже в пользу Украины росс...,Суд в Севастополе арестовал россиянина за госи...,2021-04-22 14:34:57,https://www.gazeta.ru/social/2021/04/22/135686...
497,Очная встреча президентов России и Украины дол...,Владимир Зеленский призвал Владимира Путина вс...,Зеленский пригласил Путина встретиться в Донба...,2021-04-21 11:19:13,https://www.gazeta.ru/politics/2021/04/21_a_13...
498,"В Туве, Ингушетии и Кабардино-Балкарии по итог...","В июне «РИА Рейтинг» назвал Туву, Ингушению и ...",Минтруд назвал регионы с наибольшим уровнем бе...,2021-08-24 16:49:26,https://www.gazeta.ru/social/2021/08/24/139095...


# Bart

In [7]:
bart_tokenizer = MBartTokenizer.from_pretrained(BART_MODEL_NAME)
bart_model = MBartForConditionalGeneration.from_pretrained(BART_MODEL_NAME).to(device)

In [8]:
predicts = []

for text in tqdm(dataset['test']['text'], desc = 'Тестирование'):

    input_ids = bart_tokenizer(text, 
                               max_length = MAX_LENGTH,
                               padding = PADDING,
                               truncation = TRUNCATION,
                               return_tensors = RETURN_TENSORS)
    
    input_ids = input_ids['input_ids'].to(device)
    
    output_ids = bart_model.generate(input_ids = input_ids,
                                     no_repeat_ngram_size = NO_REPEAT_NGRAM)
    
    output_ids = output_ids[0]

    predicts.append(bart_tokenizer.decode(output_ids, skip_special_tokens = True))

Тестирование: 100%|██████████| 500/500 [07:39<00:00,  1.09it/s]


In [9]:
df['summary'][10]

'Екатеринбургская епархия в исполнение решения суда прибыла на территорию Среднеурального женского монастыря, который ранее «захватил» бывший схимонах Сергий. Представители РПЦ вместе с приставами и экспертами хотели провести экспертизу с целью установления права собственности на монастырский комплекс.'

In [10]:
predicts[10]

'Представители епархии Екатеринбурга вместе с вооруженными судебными приставами прибыли в Среднеуральский женский монастырь, где укрывается отлученный от церкви экс-схимонах Сергий (в миру — Николай Романов). Сначала прихожане пытались воспрепятствовать проходу представителей епархии, но после переговоров согласились впустить на территорию монастыря как минимум эксперта. В епархии отметили, что земельные участки, входящие в комплекс монастыря, предоставлены епархии для осуществления религиозной деятельности.'

In [11]:
rouge_res = rouge.compute(predictions = predicts,
                          references = df['summary'].values)

bleu_res = bleu.compute(predictions = predicts,
                        references = df['summary'].values)

rouge_res = {key: val * 100 for key, val in rouge_res.items()}
bleu_res['bleu'] = bleu_res['bleu'] * 100

In [12]:
print(f'Rouge: {rouge_res}')
print(f'Bleu = {bleu_res["bleu"]:.2f}%')

Rouge: {'rouge1': 21.89023286654867, 'rouge2': 8.238439664910253, 'rougeL': 21.3783841304894, 'rougeLsum': 21.207524384799942}
Bleu = 8.96%


In [13]:
metrics = {}

metrics.update(rouge_res)
metrics['bleu'] = bleu_res["bleu"]

with open('bart_metrics', 'wb') as file:
    pickle.dump(metrics, file)

# T5

In [7]:
t5_model = MT5ForConditionalGeneration.from_pretrained(T5_MODEL_NAME)
t5_tokenizer = MT5Tokenizer.from_pretrained(T5_MODEL_NAME)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
def preprocess(examples):
   
    inputs = examples['text']

    model_inputs = t5_tokenizer(inputs, 
                                max_length = MAX_SOURCE_TOKENS_COUNT, 
                                truncation = True)
    
    labels = t5_tokenizer(text_target = examples['summary'], 
                          max_length = MAX_TARGET_TOKENS_COUNT,
                          truncation = True)
    
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [9]:
tokenized_dataset = dataset.map(preprocess, batched = True, desc = 'Токенезация')

Токенезация:   0%|          | 0/200 [00:00<?, ? examples/s]

Токенезация:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer = t5_tokenizer, model = t5_model)

In [11]:
def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    # Replace -100 in the labels as we can't decode them.
    predictions = np.where(predictions != -100, predictions, t5_tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)

    decoded_preds = t5_tokenizer.batch_decode(predictions, skip_special_tokens = True)
    decoded_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens = True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = {}
    result_rouge = rouge.compute(predictions = decoded_preds, references = decoded_labels)

    # Extract a few results
    result.update({key: value * 100 for key, value in result_rouge.items()})

    result_bleu = bleu.compute(predictions = decoded_preds, references = decoded_labels)

    # Extract a few results
    result["bleu"] = result_bleu["bleu"] * 100

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != t5_tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["char_len"] = np.mean([len(t) for t in decoded_preds])

    return {k: round(v, 4) for k, v in result.items()}

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir = OUTPUT_DIR,
    evaluation_strategy = EVALUATION_STRATEGY,
    eval_steps = EVAL_STEPS,
    logging_steps = LOGGING_STEPS,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size = PER_DEVICE_EVAL_BATCH_SIZE,
    gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS,
    weight_decay = WEIGHT_DECAY,
    save_total_limit = SAVE_TOTAL_LIMIT,
    num_train_epochs = NUM_TRAIN_EPOCHS,
    fp16 = FP16,
    predict_with_generate = PREDICT_WITH_GENERATE,
    generation_max_length = GENERATION_MAX_LENGTH,
    generation_num_beams = GENERATION_NUM_BEAMS
)

In [13]:
trainer = Seq2SeqTrainer(
    model = t5_model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    tokenizer = t5_tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

In [14]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

{'train_runtime': 11.1739, 'train_samples_per_second': 17.899, 'train_steps_per_second': 0.089, 'train_loss': 21.997058868408203, 'epoch': 0.64}


TrainOutput(global_step=1, training_loss=21.997058868408203, metrics={'train_runtime': 11.1739, 'train_samples_per_second': 17.899, 'train_steps_per_second': 0.089, 'train_loss': 21.997058868408203, 'epoch': 0.64})

In [15]:
eval_res = trainer.evaluate()
eval_res

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 12.623726844787598,
 'eval_rouge1': 1.0751,
 'eval_rouge2': 0.2461,
 'eval_rougeL': 1.0408,
 'eval_rougeLsum': 1.0559,
 'eval_bleu': 0.0471,
 'eval_gen_len': 13.936,
 'eval_char_len': 41.112,
 'eval_runtime': 69.5649,
 'eval_samples_per_second': 7.188,
 'eval_steps_per_second': 1.797,
 'epoch': 0.64}

In [16]:
metrics = {}

metrics['rouge1'] = eval_res['eval_rouge1']
metrics['rouge2'] = eval_res['eval_rouge2']
metrics['rougeL'] = eval_res['eval_rougeL']
metrics['rougeLsum'] = eval_res['eval_rougeLsum']
metrics['bleu'] = eval_res['eval_bleu']

with open('t5_metrics', 'wb') as file:
    pickle.dump(metrics, file)

# Сравнение метрик Bert и T5

In [17]:
metrics = []

In [18]:
with open('bart_metrics', 'rb') as file:
    metrics.append(pickle.load(file))

with open('t5_metrics', 'rb') as file:
    metrics.append(pickle.load(file))

In [19]:
metrics = pd.DataFrame(metrics, index = ['Bart', 'T5'])
metrics

,rouge1,rouge2,rougeL,rougeLsum,bleu
Bart,21.890233,8.23844,21.378384,21.207524,8.96383
T5,1.075100,0.24610,1.040800,1.055900,0.04710
